### drivers of country-pair subtype distances

In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import statsmodels.formula.api as smf
from sklearn.utils import shuffle

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
outdir = './out/'
if not os.path.exists(outdir):
    os.makedirs(outdir)

#### Load data

In [ ]:
df_lm = pd.read_csv('./input_spatial_analysis.csv')
df_lm.head(1)

### variables:
# country_i: country name of the first country, following the FluNet nomenclature
# country_j: country name of the second country, following the FluNet nomenclature
# part2gr_i: label identifying the cluster the country i belongs to, considering a 2-groups partition
# part2gr_j: label identifying the cluster the country j belongs to, considering a 2-groups partition
# part6gr_i: label identifying the cluster the country i belongs to, considering a 6-groups partition
# part6gr_j: label identifying the cluster the country j belongs to, considering a 6-groups partition
# d_subtypes: distances of trajectories of (sub)type compositions. Trajectories are expressed in ilr coordinates, and cover the 
#    period 2010-2019. Distances are quantified in terms of average Euclidean distances between corresponding points of the 2 trajectories.
# d_flights: air-traffic distances in terms of arrival time. This variable is hidden as IATA data are not shareable.
# d_T: difference of average temperatures
# d_RH: difference of relative humidities
# season_sync: categorical variable, specifying whether the 2 countries have synchronized, partially overlapping, or oppsite flu seasons.

# for a detailed explanation of the variable definitions look at the Supplementary Information of the paper
# Characterization and forecast of global influenza (sub)type dynamics doi: https://doi.org/10.1101/2024.08.01.24311336

#### Drivers of country subtype dissimilarities - multivariate analysis

Transform the variables to make their distributions more symmetric and standardize

In [ ]:
### transform the numerical variables to make them more Gaussian
df_lm_std = df_lm.copy()
df_lm_std['d_subtypes'] = np.log(df_lm_std['d_subtypes'])
df_lm_std['d_T'] = np.sqrt(df_lm_std['d_T'])
df_lm_std['d_RH'] = np.sqrt(df_lm_std['d_RH'])
# d_flights is not transformed

### standardize the numerical variables
numeric_cols = ['d_subtypes', 'd_flights', 'd_T', 'd_RH']
scaler = StandardScaler()
df_lm_std[numeric_cols] = scaler.fit_transform(df_lm_std[numeric_cols])

### visualize distributions, first and after the transformation
fig, axs = plt.subplots(2,4,figsize=(6,3))
axs = axs.ravel()

new_vars = ['log(d_subtypes)', 'd_flights', 'sqrt(d_T)', 'sqrt(d_RH)']
for i,var in enumerate(numeric_cols):

    ### original variables
    ax = axs[i]
    ax.hist(df_lm[var], bins=30)#, label='%s'%var)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(var, fontsize=8)

    ### transformed variables
    ax = axs[i+4]
    ax.hist(df_lm_std[var], bins=30)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(new_vars[i], fontsize=8)

fig.tight_layout()   

Variance Inflation Factor (VIF) for Multicollinearity. A VIF>5 suggests a multicollinearity problem


In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Compute VIF for each variable
vif_data = pd.DataFrame()
vif_data["Variable"] = numeric_cols
vif_data["VIF"] = [variance_inflation_factor(df_lm_std[numeric_cols].values, i) for i in range(len(numeric_cols))]

print(vif_data)

Multivariate Linear Model with permutation to estimate parameters' confidence intervals (see: https://doi.org/10.1007/s11258-006-9126-3)

In [ ]:
lm_model = smf.ols(formula='d_subtypes ~ C(season_sync) + d_flights + d_T + d_RH', data=df_lm_std).fit()
estimated_coefs = lm_model.params
#print ('d_subtypes ~ C(season_sync) + d_flights + d_T + d_RH')
#lm_model.summary()

In [ ]:
### fit linear models on permuted data
n_permutations = 10000
permuted_coefs = np.zeros((n_permutations, len(estimated_coefs)))
for i in range(n_permutations):
    Y_permuted = shuffle(df_lm_std['d_subtypes'])
    df_lm_std['d_subtypes_permuted'] = list(Y_permuted)
    perm_model = smf.ols(formula='d_subtypes_permuted ~ C(season_sync) + d_flights + d_T + d_RH', data=df_lm_std).fit()
    permuted_coefs[i, :] = perm_model.params

### compute the empirical confidence intervals
lower_bound = np.percentile(permuted_coefs, 0.05, axis=0)
upper_bound = np.percentile(permuted_coefs, 99.95, axis=0)
pvalues = []
for idx, (var,coef) in enumerate(estimated_coefs.items()):
    distrib = permuted_coefs[:,idx]
    pval = len(distrib[distrib<coef])/n_permutations
    if pval>0.5:
        pval = 1-pval
    pvalues.append(pval)
    #print(f"{var}: {coef:.3f} \t pval={pval:.3f} \t(99,9% CI: {lower_bound[idx]:.3f}, {upper_bound[idx]:.3f}) ")
    
### store results of the linear model
df_lm_res = pd.DataFrame(columns=['variable','coefficient','pval'])
df_lm_res['variable'] = [k for k,v in estimated_coefs.items()]
df_lm_res['coefficient'] = [v for k,v in estimated_coefs.items()]
df_lm_res['pval'] = pvalues
df_lm_res